In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, date

In [ ]:
b3stocks_T1_path = "Bolsa_de_valores/transformacoes/b3stocks_T1.csv"
b3_stocks_1994_2020_path = "Bolsa_de_valores/arquivo_base/b3_stocks_1994_2020.csv"

In [ ]:
b3_stocks = pd.read_csv(b3_stocks_1994_2020_path, header='infer')
b3stocks_T1 = pd.read_csv(b3stocks_T1_path, header='infer')


Transformar data (datetime) para DD/MM/AAAA. Atualmente o formato está YYY-MM-DD. 1994-07-04
Gerar um arquivo chamado b3stocks_T1.csv

In [ ]:
b3_stocks['datetime'] = pd.to_datetime(b3_stocks.datetime, format='%Y-%m-%d')
b3_stocks['data_formatada'] = b3_stocks['datetime'].dt.strftime('%d/%m/%Y')
datas_formatadas_aluno_t1 = b3stocks_T1['datetime']

In [ ]:
print(f"Datas vazias no df original: {b3_stocks['datetime'].isnull().sum()}")
print(f"Datas vazias no df formatado: {datas_formatadas_aluno_t1.isnull().sum()}")

In [ ]:
pd.testing.assert_series_equal(b3_stocks['data_formatada'], datas_formatadas_aluno_t1, check_names=False)

# T2 Considerando o arquivo anterior, fazer uma filtragem para que fique apenas um registro por dia. Esse registro deve ser apenas quele que possuir o maior volume negociado em bolsa.

In [ ]:
b3_stocks.info()

In [ ]:
b3_stocks.reset_index(inplace=True)

In [ ]:
# maior_valor_por_dia = b3_stocks.groupby(['datetime','ticker'])['volume'].max()
maior_valor_por_dia = b3_stocks.loc[b3_stocks.groupby('datetime')['volume'].idxmax()]
# maior_valor_por_dia.columns = ['datetime', 'ticker', 'volume']
# maior_valor_por_dia.reset_index(inplace=True)
# maior_valor_por_dia.to_csv("Bolsa_de_valores/transformacoes/maior_valor_por_dia.csv", header=True)
b3stocks_filtro_T2_path = 'Bolsa_de_valores/transformacoes/b3stocks_F1.csv'
b3stocks_filtro_T2 =  pd.read_csv(b3stocks_filtro_T2_path, header='infer')
b3stocks_filtro_T2['datetime'] = pd.to_datetime(b3stocks_filtro_T2['datetime'], format='%d/%m/%Y')

In [ ]:
# print(b3stocks_filtro_T2[['datetime','volume']])
# maior_valor_por_dia.columns.values[1] = 'volume'
# print(maior_valor_por_dia.columns)
# 
display(maior_valor_por_dia)

# print(b3stocks_filtro_T2[['datetime','volume']])

In [ ]:
# maior_valor_por_dia[['datetime','volume']].info()
b3stocks_filtro_T2[['datetime','volume']].info()

In [ ]:
maior_valor_por_dia.merge( b3stocks_filtro_T2, indicator=True, how='outer')
# pd.testing.assert_series_equal(maior_valor_por_dia[['datetime','volume']], b3stocks_filtro_T2[['datetime','volume']], check_names=False)

Considerando o arquivo da transformação T1 (b3stocks_T1.csv), fazer uma filtragem para que fique apenas os registros que possuírem volume negociado acima da média diária.

In [ ]:
# Calcula a média diária de volume
media_diaria = b3_stocks.groupby('datetime')['volume'].mean()

# Cria uma coluna com a média diária
b3_stocks['media_diaria'] = b3_stocks['datetime'].map(media_diaria)
b3_stocks['media_diaria'] = b3_stocks['media_diaria'].apply(float)
# Filtra as linhas com volume acima da média diária
b3stocks_Media_diaria_maior = b3_stocks[b3_stocks['volume'] > b3_stocks['media_diaria']]

In [ ]:
b3stocks_Media_diaria_maior.info()

In [ ]:
b3_stocks['media_diaria'] = b3_stocks['media_diaria'].map('{:.2f}'.format)

In [ ]:
display(b3stocks_Media_diaria_maior.sort_values('datetime', ascending=True))

In [ ]:
b3stocks_Filtro_T3_path = "Bolsa_de_valores/transformacoes/b3stocks_T1_Filtrado.csv"
b3stocks_filtro_T3 = pd.read_csv(b3stocks_Filtro_T3_path, header='infer')
b3stocks_filtro_T3['datetime'] = pd.to_datetime(b3stocks_filtro_T3['datetime'], format='%d/%m/%Y')


In [ ]:
resultado = b3stocks_Media_diaria_maior.merge(b3stocks_filtro_T3, indicator=True, how='outer')

In [ ]:
print("Total de registros filtrados: {:,}".format(len(b3stocks_Media_diaria_maior)))
print("Total de registros filtrados pelo grupo: {:,}".format(len(b3stocks_filtro_T3)))
print("Total de registros filtrados diferença: {:,}".format(len(resultado[resultado['_merge'] != 'both'])))


In [ ]:
# Remove a coluna auxiliar
# df_filtrado = df_filtrado.drop('media_diaria', axis=1)